## Questão 06
Pesquise sobre redes neurais recorrentes LSTM. Apresente neste estudo aplicações das 
LSTM deep learning. Seguem abaixo sugestões de aplicações.

***i) Processamento de Linguagem Natural***

ii) Conversão voz/texto

iii) Tradução de textos

iv) outras aplicações


- Utilizando o tema de Processamento de Linguagem Natural, iremos analisar uma aplicação que faz uso do LSTM deep learning que faz a previsão de quatro categorias de problemas de saúde mental: depressão, alcoolismo, drogas e suicídio.

- O dataset desse projeto é composto por declarações e perguntas expressas por estudantes de diversas universidades do Quênia, onde relataram sofrer com esses desafios de saúde mental.

- O projeto está disponível no seguinte [link](https://dadosaocubo.com/processamento-de-linguagem-natural-com-tensorflow/).

#Carregando Bibliotecas e Dataset
Primeiramente vamos começar importando todas as bibliotecas para rodar o nosso modelo. Destaque para as biblioteca Pandas, Numpy, NLTK e TensorFlow! Pandas utilizamos para importar e organizar os nossos dados em formato de data frame. Numpy utilizamos para fazer algumas conversões importantes com nossos dados. NLTK utilizamos para trabalhar com o texto, muito poderosa para problemas de NLP. Por fim o TensorFlow que utilizamos para o nosso modelo de machine learning. As demais bibliotecas foram utilizadas de forma auxiliar e vou te explicar no decorrer do código.

# Importação das bibliotecas

In [ ]:
!pip install wordninja >log

In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import wordninja
import textblob
import collections
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Após as bibliotecas importadas vamos carregar o dataset para começar a trabalhar os dados.

In [ ]:
# Importação dos dataset
df = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/nlp/master/mental_health.csv')

#Preparação dos Dados

Vamos começar a preparar os dados para serem consumidos pelo modelo. Então, criamos algumas variáveis que vamos utilizar mais a frente. A lista de palavras vazias (stop_words), o tamanho da base para teste (test_size) e duas variáveis para o modelo quantidade de épocas (epochs) e número de exemplos por lote (batch_size).

In [ ]:
# Palavras para retirar da análise
stop_words = stopwords.words('english')
# Variável tamanho da base de teste
test_size = 0.1
# Variáveis do modelo
epochs = 10
batch_size = 128

Agora vamos começar a tratar os nosso dados, fazer aquela faxina. Dessa forma o “lixo” que vamos tirar, não vai atrapalhar o nosso modelo.

#Limpeza dos Dados

O primeiro passo separar as possíveis palavras concatenadas. Para essa tarefa criamos a coluna text_split e aplicamos a função split da biblioteca wordninja. Na sequência criamos a coluna text_new e aplicamos a função TreebankWordDetokenizer().detokenize para formar novamente um texto com as palavras separadas.

In [ ]:
# Separando palavras juntas
df['text_split'] = df['text'].apply(wordninja.split)
df['text_new'] = df['text_split'].apply(TreebankWordDetokenizer().detokenize)

Agora vamos corrigir palavras escritas de forma incorreta. Dessa vez vamos aplicar a função textblob.TextBlob.correct da biblioteca textblob na mesma coluna text_new . Observe que já existe muita coisa pronta em python, basta saber encontrar para utilizar.

In [ ]:
# Corrigindo palavras incorretas
df['text_new'] = df['text_new'].apply(textblob.TextBlob).apply(textblob.TextBlob.correct).apply(str)

Para o tratamento de números, pontuação e caracteres especiais utilizamos expressões regulares sempre aplicando na coluna text_new. No fim deste bloco aplicamos a função lower para deixar todas as letras e caixa baixa.

In [ ]:
# Excluindo da descrição os números.
df['text_new'] = df['text_new'].str.replace('[0-9]+', '', regex=True).copy()
# Excluindo da descrição pontuação.
df['text_new'] = df['text_new'].str.replace('[,.:;!?]+', ' ', regex=True).copy()
# Excluindo da descrição caracteres especiais.
df['text_new'] = df['text_new'].str.replace('[/<>()|\+\-\$%&#@\'\"]+', ' ', regex=True).copy()
# Colocando todos os caracteres em caixa baixa.
df['text_new'] = df['text_new'].str.lower().copy()

Por fim, na limpeza de dados vamos remover as palavras vazias, mais conhecidas como stop words. Para essa tarefa, criei a função tokenize_df (Faltou um pouco de criatividade no nome :-)) para remover as palavras vazias e retornar somente o conteúdo útil do texto.

In [ ]:
# Função para retirar stop words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stop_words]
  text = TreebankWordDetokenizer().detokenize(stop)
  return text
# Eliminando as stop words
df['text_new'] = df['text_new'].apply(tokenize_df).copy()

A cada etapa dessa realizada acima, podemos exibir as primeiras linhas do dataframe com a função head para ver se cada etapa está sendo feita corretamente. No notebook que que vou disponibilizar no final foi feito essa análise.

Ao final dessa limpeza estamos pronto para o input no modelo? Não! Precisamos transformar nossa coluna text_new em objetos do tipo tensor para que o modelo entenda os dados. Vamos ver como fazer essa transformação!

#Transformação para Input no Modelo
Vamos começar gerando uma lista de palavras únicas de todas as declarações e perguntas das base após a limpeza.

In [ ]:
# Selecionando as únicas palavras da variável text_new
df['text_new_split'] = df['text_new'].apply(word_tokenize).copy()
text = list(df.text_new_split)
list_words = [item for sublist in text for item in sublist]
list_words = sorted(list_words)
only_words = set(list_words)

Com o código acima criamos uma lista com todas as palavras na variável list_words e com a função set removemos as palavras duplicadas gerando a variável only_words apenas com as palavras únicas.

Então, vamos gerar o nosso codificador de palavras com o vocabulário de palavras únicas q criamos assim. Para isso vamos usar a função SubwordTextEncoder(vocab_list=only_words), passando como parâmetro a nossa lista de palavras únicas only_words.

In [ ]:
# Gerando encoder com o vocabulário das palavras
encoder = tfds.deprecated.text.SubwordTextEncoder(vocab_list=only_words)

O encoder acima foi para a nossa feature, ou seja, nossa variável de entrada. Precisamos agora fazer o codificador do target ou nossa variável de saída. Vamos usar a função do scikit-learn LabelEncoder, como pode ver no código abaixo.

In [ ]:
# Encode do label
label_encode = LabelEncoder()
target = label_encode.fit_transform(df['label'])

Agora precisamos definir as nossas variáveis de feature e target, como podemos ver no código abaixo representadas como x e y. Ainda usando o scikit-learn com a função train_test_split dividimos os dados em treino e teste para fazer uma avaliação de performance do modelo. Lembra da variável test_size que definimos lá no ínicio do item 4.? Agora vamos usar ela para definir o tamanho dos dados de teste.

In [ ]:
# Definindo feature e target
x = df['text_new']
y = target
# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)

Por fim na transformação do input, vamos converter os dados de entradas para o formato de tensor.

Para essa etapa crie duas funções. A função pad_to_size que define o tamanho do vetor e complementa com zeros. E a função encode_input que pega o texto, converte em vetor usando a função encoder (que falei um pouco mais acima) completa o vetor com zeros com a função pad_to_size e por fim transforma esse vetor em um tensor com a função tf.cast.

In [ ]:
# Função para criação da matriz
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec
# Função para encode do input
def encode_input(text_new):
  list_x = []
  for text in text_new:
    text_encode = encoder.encode(text)
    text_encode = pad_to_size(text_encode, 64)
    list_x.append(text_encode)
  # Convertendo x em tensor
  input_encode = tf.cast(list_x, tf.float32)
  return input_encode
# Encode do x_train e x_test
x_train = encode_input(x_train)
x_test = encode_input(x_test)

Então saímos com os dados prontos para o input no modelo. Agora vamos ver como criar esse modelo.

#Modelo do TensorFlow
Para a resolução do problema vamos utilizar uma Rede Neural Recorrente. Agora preciso em poucas palavras te dizer o que é uma RNN.

Para definir uma RNN vamos ver a definição encontrada no Machine Learning Glossary do Google, adaptada e traduzida:

“Uma rede neural que é executada intencionalmente várias vezes, em que partes de cada execução alimentam a próxima execução. Especificamente, as camadas ocultas da execução anterior fornecem parte da entrada para a mesma camada oculta na execução seguinte. As redes neurais recorrentes são particularmente úteis para avaliar sequências, para que as camadas ocultas possam aprender com as execuções anteriores da rede neural em partes anteriores da sequência.”

De uma forma bruta é uma rede que tem como entrada nas camadas ocultas a própria saída de forma intencional até encontrar a equação matemática desejada.

As redes RNN são boas em resolver problemas de NLP, então criamos uma com 6 camadas, e vamos detalhar um pouco mais essas camadas a seguir.

In [ ]:
# Criação do modelo de RNN
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(encoder.vocab_size, 128)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         123904    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 4)                 516   

Com a função summary é possível ter um resumo do modelo criado como podemos ver na imagem acima.

#Input no TensorFlow
É uma camada obrigatória em todos os modelos, é a camada que recebe os dados de entrada.

#Embedding no TensorFlow
É a camada onde é feita a tradução do vetor de entrada em alta dimensão para uma para um espaço de baixa dimensão. 

#LSTM no TensorFlow
LSTM é a abreviação de Long Short-Term Memory em tradução livre Memória Longa de Curto Prazo. É a camada responsável por armazenar em memória a informação das camadas anteriores, mantendo um histórico das novas entradas para melhorar a performance do modelo. 

#Dropout no TensorFlow
Esta é uma camada de regularização da rede, que aleatoriamente exclui um número fixo de unidades em uma camada, para forçar uma regularização do modelo.

#Dense no TensorFlow
É a nossa camada de saída, a qual está conectadas a todos os nós das camada ocultas anteriores.

#Treinamento do Modelo
Com o nosso modelo pronto, podemos compilar com a função compile. Onde é definido a função de perda com o parâmetro loss, a função de otimização com o parâmetro optimizer e a métrica com o parâmetro metrics.

Compilado, vamos treinar o nosso modelo!

In [ ]:
# Compilando modelo e configurando o processo de treinamento
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=['accuracy'])
# Treinando o modelo
history = model.fit(x_train, y_train, epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_test, y_test))

Epoch 1/10
5/5 [==============================] - 12s 936ms/step - loss: 1.2637 - accuracy: 0.4801 - val_loss: 1.2037 - val_accuracy: 0.5323
Epoch 2/10
5/5 [==============================] - 3s 546ms/step - loss: 1.1276 - accuracy: 0.5758 - val_loss: 1.2003 - val_accuracy: 0.5323
Epoch 3/10
5/5 [==============================] - 2s 435ms/step - loss: 1.1232 - accuracy: 0.5758 - val_loss: 1.1842 - val_accuracy: 0.5323
Epoch 4/10
5/5 [==============================] - 2s 414ms/step - loss: 1.1164 - accuracy: 0.5758 - val_loss: 1.2081 - val_accuracy: 0.5323
Epoch 5/10
5/5 [==============================] - 2s 435ms/step - loss: 1.1038 - accuracy: 0.5758 - val_loss: 1.1618 - val_accuracy: 0.5323
Epoch 6/10
5/5 [==============================] - 2s 429ms/step - loss: 1.0976 - accuracy: 0.5758 - val_loss: 1.1273 - val_accuracy: 0.5323
Epoch 7/10
5/5 [==============================] - 4s 717ms/step - loss: 1.0476 - accuracy: 0.5758 - val_loss: 1.0520 - val_accuracy: 0.6613
Epoch 8/10
5/5 [===

As variáveis epochs e batch_size que definimos no início do item 4., usamos no nosso modelo para treinamento. Abaixo o resultado do treinamento do modelo ao final das 10 épocas definidas.

#Avaliação do Modelo
Para avaliar o resultado do nosso modelo utilizei a função evaluate, que retorna a função de perda e a acurácia do nosso modelo.

In [ ]:
# Testando a qualidade do modelo
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

2/2 [==============================] - 0s 41ms/step - loss: 0.8283 - accuracy: 0.6129
Test Loss: 0.8283111453056335
Test Accuracy: 0.6129032373428345


#Predição com o Modelo
Enfim o modelo está pronto, treinado e avaliado! Para testar novas entradas fiz a função sample_predict que retorna a probabilidade de cada uma das 4 classes.

In [ ]:
# Função para predição
def sample_predict(sample_pred_text):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))
  return (predictions)

Com as probabilidades de cada classe, usamos a função argmax do numpy para retornar a posição com a probabilidade mais alta. Dessa forma podemos usar essa posição na função inverse_transform que retorna o label da nossa classe. Podemos ver isso no código abaixo.

In [ ]:
# Predição do exemplo
example = 'feel better die happy'
predictions = sample_predict(example)
probabilities = [np.argmax(predictions[0])]
# Retornando os labels
new_label = label_encode.inverse_transform(probabilities)
print('O exemplo "{}" foi classificado como "{}"'.format(example, new_label[0]))

1/1 [==============================] - 2s 2s/step
O exemplo "feel better die happy" foi classificado como "Depression"


In [ ]:
# Predição do exemplo
example = 'drugs beer drink'
predictions = sample_predict(example)
probabilities = [np.argmax(predictions[0])]
# Retornando os labels
new_label = label_encode.inverse_transform(probabilities)
print('O exemplo "{}" foi classificado como "{}"'.format(example, new_label[0]))

1/1 [==============================] - 0s 39ms/step
O exemplo "drugs beer drink" foi classificado como "Alcohol"


In [ ]:
# Predição do exemplo
example = 'no one will miss me'
predictions = sample_predict(example)
probabilities = [np.argmax(predictions[0])]
# Retornando os labels
new_label = label_encode.inverse_transform(probabilities)
print('O exemplo "{}" foi classificado como "{}"'.format(example, new_label[0]))

1/1 [==============================] - 0s 29ms/step
O exemplo "no one will miss me" foi classificado como "Depression"
